<a href="https://colab.research.google.com/github/2gauravc/random_notebooks/blob/main/Hanane_Apr24_RAG_for_Fin_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index-llms-anthropic
!pip install llama-index-embeddings-openai
!pip install llama-index
!pip install anthropic
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.0/248.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.7 MB/s eta 0:00:00


In [ ]:
 from llama_index.llms.anthropic import Anthropic
 from llama_index.core.tools import FunctionTool
 import nest_asyncio
 nest_asyncio.apply()
 from google.colab import userdata

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

CLAUDE_API_KEY = userdata.get('CLAUDE_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
LLAMA_CLOUD_API_KEY = userdata.get('LLAMA_CLOUD_API_KEY')

import openai
openai.api_key = OPENAI_API_KEY

print("API Keys loaded successfully!")


API Keys loaded successfully!


1.1 Load Documents

In [ ]:
# Path to your API keys file
pdf_file = '/content/drive/My Drive/Colab Notebooks/jpm_annualreport-2022_splitted.pdf'

2.0 RAG

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_parse import LlamaParse

# set up parser
parser = LlamaParse(api_key=LLAMA_CLOUD_API_KEY, result_type="markdown")
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=[pdf_file],file_extractor=file_extractor).load_data()
# Index the documents in a vectorStore
index_jpmorgan = VectorStoreIndex.from_documents(documents)

Started parsing the file under job_id 85bdb66a-3111-409b-b05b-9deb4e368801


In [ ]:
len(documents)

5

2.2  Query the Document using GPT-4-Turbo

In [ ]:
from llama_index.llms.openai import OpenAI
llm_openai = OpenAI(model="gpt-4-turbo", temperature=0.1)
query_engine_jpmorgan = index_jpmorgan.as_query_engine(llm=llm_openai)
# query the engine
query = "What was the Net Income in 2020, the Reported one?"
response_jpmorgan = query_engine_jpmorgan.query(query)
print("*"*50)

print(f"The answer to the question '''{query}''' is :\n")

response_jpmorgan.response

**************************************************
The answer to the question '''What was the Net Income in 2020, the Reported one?''' is :



'The net income for JPMorgan Chase in 2020 was $29,131 million.'

3. Build an Anthropic Agent over RAG

3.1 RAG Tool

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
#Create the Query Engine Tool over the RAG pipeline
query_engine_tool = QueryEngineTool(
  query_engine=query_engine_jpmorgan,
  metadata=ToolMetadata(
  name="jpmorgan_annualreport-2022",
  description=(
    "Provides Annual Report information about JPMorgan for 2022. "
    "Use a detailed plain text question as input to the tool."
    ),
  ),)

 Create the FunctionCallingAgentWorker using the query engine tool and initialize the Anthropic Agent. Here we are using only one tool, which is the one built over the RAG pipeline.

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
llm_anthropic = Anthropic(model="claude-3-opus-20240229", api_key=CLAUDE_API_KEY)
agent_worker = FunctionCallingAgentWorker.from_tools(
 [query_engine_tool], llm=llm_anthropic, verbose=True)
agent = agent_worker.as_agent()

3.2 Chat with the Agent

In [ ]:
query = "What was the Net Income in 2020, the Reported one?"
response = agent.chat(query)
print(str(response))

Added user message to memory: What was the Net Income in 2020, the Reported one?
=== LLM Response ===
<thinking>
To answer the question about JPMorgan's Net Income in 2020, the jpmorgan_annualreport-2022 tool seems relevant. It provides information from JPMorgan's 2022 annual report, which likely includes financial data from prior years like 2020.

The tool requires an "input" parameter with a detailed plain text question. The user's question is already a detailed plain text question asking for a specific financial metric (Net Income) for a specific year (2020). No additional information seems needed to use this tool to attempt to answer the question.
</thinking>
=== Calling Function ===
Calling function: jpmorgan_annualreport-2022 with args: {"input": "What was JPMorgan's reported Net Income in 2020?"}
=== Function Output ===
JPMorgan's reported Net Income in 2020 was $29,131 million.
=== LLM Response ===
Based on the information provided by the jpmorgan_annualreport-2022 tool, JPMorg

In [ ]:
query = "What was the evolution of the Net Income from 2020 to 2021, the Reported one?"
response = agent.chat(query)
print(str(response))

Added user message to memory: What was the evolution of the Net Income from 2020 to 2021, the Reported one?
=== LLM Response ===
<thinking>
To answer the question about the evolution of JPMorgan's reported Net Income from 2020 to 2021, the jpmorgan_annualreport-2022 tool remains relevant. It should contain the Net Income figures for both 2020 and 2021 to allow calculating the change between those two years.

The tool requires a detailed plain text question as input. To get the necessary information, I will ask for the Net Income values for each year separately. By having both the 2020 and 2021 Net Income numbers, I can then calculate how the Net Income evolved over that period.
</thinking>
=== Calling Function ===
Calling function: jpmorgan_annualreport-2022 with args: {"input": "What was JPMorgan's reported Net Income in 2020?"}
=== Function Output ===
JPMorgan's reported Net Income in 2020 was $29,131 million.
=== Calling Function ===
Calling function: jpmorgan_annualreport-2022 with

In [ ]:
query = "What was the evolution in term of percentage of the Net Income from 2020 to 2021, the Reported one?"
response = agent.chat(query)
print(str(response))

Added user message to memory: What was the evolution in term of percentage of the Net Income from 2020 to 2021, the Reported one?
=== LLM Response ===
<thinking>
The previous response already calculated the percentage change in JPMorgan's reported Net Income from 2020 to 2021. The necessary Net Income values for 2020 and 2021 were obtained using the jpmorgan_annualreport-2022 tool.

2020 Net Income: $29,131 million
2021 Net Income: $48,334 million

The percentage change calculation was:

Percent change = (2021 Net Income - 2020 Net Income) / 2020 Net Income * 100
               = ($48,334 million - $29,131 million) / $29,131 million * 100
               = 66.0% increase

Since the percentage evolution has already been determined using the data from the tool, no additional information or calculations are needed to answer this question.
</thinking>

Based on the Net Income data provided by the jpmorgan_annualreport-2022 tool, JPMorgan's reported Net Income increased by 66.0% from 2020 to

3.3 Add another tool to the agent

In [ ]:
def evolution(a: float, b: float)-> float:
 """Evolution in percentage from value b to value a"""
 return f"{round(100*(a/b-1),0)}%"
evolution_tool = FunctionTool.from_defaults(fn=evolution)

In [ ]:
 def cagr(a: float, b: float, n: int)-> float:
  """Compound annual growth rate CAGR from value b to value a over n period"""
  return f"{round(100*((a/b)^(1/n)-1),0)}%"
cagr_tool = FunctionTool.from_defaults(fn=cagr)

In [ ]:
def price_earning_ratio(price: float, eps: float)-> float:
 """Price per earning ratio is a ratio of the share price devided by the␣ earning per share(EPS) value"""
 return price/eps
pe_tool = FunctionTool.from_defaults(fn=price_earning_ratio)

3.4 Update the agent with the second tool

In [ ]:
agent_worker2 = FunctionCallingAgentWorker.from_tools(
 [query_engine_tool, evolution_tool], llm=llm_anthropic, verbose=True)
agent2 = agent_worker2.as_agent()
query = "What was the evolution of the Net Income from 2020 to 2021, the Reported one?"
response = agent2.chat(query)
print(str(response))

Added user message to memory: What was the evolution of the Net Income from 2020 to 2021, the Reported one?
=== LLM Response ===
<thinking>
To calculate the evolution of JPMorgan's Net Income from 2020 to 2021, I will need the Net Income values for those two years. 

The jpmorgan_annualreport-2022 tool can likely provide the Net Income information, but I need to ask it a clear question to get the specific values I need.

Once I have the 2020 and 2021 Net Income numbers, I can pass them to the evolution tool to calculate the percentage change between the two years.
</thinking>
=== Calling Function ===
Calling function: jpmorgan_annualreport-2022 with args: {"input": "What was JPMorgan's reported Net Income in 2020 and 2021?"}
=== Function Output ===
JPMorgan's reported net income for 2020 was $29,131 million and for 2021 it was $48,334 million.
=== LLM Response ===
<thinking>
The jpmorgan_annualreport-2022 tool provided the Net Income values I needed:
2020: $29,131 million 
2021: $48,33

In [ ]:
query = "What was the evolution of the Net Income from 2021 to 2022, the Reported one?"
response = agent2.chat(query)
print(str(response))

Added user message to memory: What was the evolution of the Net Income from 2021 to 2022, the Reported one?
=== LLM Response ===
<thinking>
To calculate the evolution of JPMorgan's Net Income from 2021 to 2022, I will need the Net Income values for those two years.

I already have the 2021 Net Income value of $48,334 million from the previous query. 

The jpmorgan_annualreport-2022 tool can provide the 2022 Net Income information. I will ask it a targeted question to get that specific value.

Once I have the 2022 Net Income number, I can pass it along with the 2021 value to the evolution tool to calculate the percentage change between the two years.
</thinking>
=== Calling Function ===
Calling function: jpmorgan_annualreport-2022 with args: {"input": "What was JPMorgan's reported Net Income in 2022?"}
=== Function Output ===
JPMorgan's reported Net Income in 2022 was $37.7 billion.
=== LLM Response ===
<thinking>
The jpmorgan_annualreport-2022 tool provided the 2022 Net Income value of

In [ ]:
query = "Compare the evolution of the Net Income between 2020 to 2021 and 2021 to 2022, the Reported one."
response = agent2.chat(query)
print(str(response))

Added user message to memory: Compare the evolution of the Net Income between 2020 to 2021 and 2021 to 2022, the Reported one.
=== LLM Response ===
<thinking>
To compare the evolution of JPMorgan's reported Net Income between the periods 2020 to 2021 and 2021 to 2022, I will use the Net Income values and evolution percentages calculated in the previous two queries.

From 2020 to 2021:
2020 Net Income: $29,131 million
2021 Net Income: $48,334 million 
Evolution: 66.0% increase

From 2021 to 2022:
2021 Net Income: $48,334 million
2022 Net Income: $37,700 million
Evolution: 22.0% decrease

I can compare the two evolution percentages to highlight the significant difference in Net Income growth between the two periods. The first period saw strong growth while the second period experienced a notable decline.
</thinking>

<result>
There is a stark contrast in the evolution of JPMorgan's reported Net Income between the periods 2020 to 2021 and 2021 to 2022:

From 2020 to 2021, Net Income grew 

In [ ]:
agent_worker3 = FunctionCallingAgentWorker.from_tools(
 [query_engine_tool, evolution_tool, cagr_tool], llm=llm_anthropic, verbose=True)
agent3 = agent_worker3.as_agent()
query = "What was the compound annual growth of the Net Income from 2020 to 2022, the Reported one?"
response = agent3.chat(query)
print(str(response))

Added user message to memory: What was the compound annual growth of the Net Income from 2020 to 2022, the Reported one?
=== LLM Response ===
<thinking>
To calculate the compound annual growth rate (CAGR) of JPMorgan's Net Income from 2020 to 2022, I will need 3 pieces of information:

1. Net Income value in 2022 (a)
2. Net Income value in 2020 (b) 
3. Number of years between 2020 and 2022 (n)

I can use the jpmorgan_annualreport-2022 tool to try to find the Net Income values for 2022 and 2020. The number of years is simply 2022 - 2020 = 2.

So the plan is:
1. Use jpmorgan_annualreport-2022 to find 2022 Net Income 
2. Use jpmorgan_annualreport-2022 to find 2020 Net Income
3. Calculate CAGR using the cagr tool with the Net Income values and n=2
</thinking>
=== Calling Function ===
Calling function: jpmorgan_annualreport-2022 with args: {"input": "What was JPMorgan's reported Net Income in 2022?"}
=== Function Output ===
JPMorgan's reported Net Income in 2022 was $37.7 billion.
=== Calli

In [ ]:
agent_worker3 = FunctionCallingAgentWorker.from_tools(
 [query_engine_tool, evolution_tool, cagr_tool, pe_tool], llm=llm_anthropic, verbose=True)
agent3 = agent_worker3.as_agent()
query = "knowning that the price of JPMorgan in 2022 was $134.10 and given the financial report, what was the price/earning ratio?"
response = agent3.chat(query)
print(str(response))

Added user message to memory: knowning that the price of JPMorang in 2022 was $134.10 and given the financial report, what was the price/earning ratio?
=== LLM Response ===
<thinking>
To calculate the price/earning ratio, we need two pieces of information:
1. The price of JPMorgan stock in 2022, which is provided as $134.10
2. The earnings per share (EPS) for JPMorgan in 2022, which is not directly provided but can likely be found in the 2022 annual report

The relevant tool to get the EPS value seems to be:
jpmorgan_annualreport-2022

This tool takes a plain text question as input. To get the EPS, we can ask it something like "What was JPMorgan's diluted earnings per share for 2022?"

Once we have the EPS value, we can then use the price_earning_ratio tool, providing the given price of $134.10 and the EPS value from the annual report.

So the plan is:
1. Call jpmorgan_annualreport-2022 to get the 2022 EPS 
2. Call price_earning_ratio with the $134.10 price and EPS from step 1
</thinki